# Run `folktexts` benchmark with a custom ACS task

This notebook describes how to define a custom prediction task based on 
American Community Survey (ACS) data.

In [1]:
import folktexts
folktexts.__version__

'0.0.18'

In [2]:
from pathlib import Path

**Note:** Change the `DATA_DIR` variable to the ACS data path on your file system (if any, otherwise will attempt to download data).

In [3]:
DATA_DIR = Path("/fast/groups/sf") / "data"

### Define custom task objects appropriately:

In this example, we'll try to predict whether someone is or ever was part of the Military, based on a series of demographic features.

We must define objects of the following classes:
- `Threshold`: Used to binarize the target column (not needed if target is already binary);
- `MultipleChoiceQA`: Defines a question and answer scheme for the target column;
- `TaskMetadata`: Defines which columns to use as features, and which to use as the prediction target;

Select the target column (among those defined in `folktexts.acs.acs_columns`):

In [4]:
TARGET_COLUMN = "MIL"

Define threshold to be applied to target column:

In [5]:
from folktexts.threshold import Threshold
target_threshold = Threshold(4, "!=")

# data["MIL"] != 4 means "Is on active duty in the military or was in the past"
# data["MIL"] == 4 means "Never served in the military"

Define question and answer interface used for prompting:

In [6]:
from folktexts.qa_interface import MultipleChoiceQA
from folktexts.qa_interface import Choice

target_column_qa = MultipleChoiceQA(
    column=target_threshold.apply_to_column_name(TARGET_COLUMN),
    text="Has this person ever served in the military?",
    choices=(
        Choice("Yes, this person is now on active duty or was in the past", 1),
        Choice("No, this person has never served in the military", 0),
    ),
)

Define task metadata:

In [7]:
from folktexts.acs import ACSTaskMetadata

task = ACSTaskMetadata.make_task(
    name="ACSMilitary",
    description="predict if a person has ever served in the military",
    features=[
        "AGEP", "SCHL", "MAR", "POBP", "WKHP", "SEX", "RAC1P", "ST", "CIT", "DIS", "PINCP",
    ],
    target=TARGET_COLUMN,
    target_threshold=target_threshold,
    sensitive_attribute="RAC1P",
    qa_interface=target_column_qa,
)

Load ACS dataset for the custom task:

In [8]:
%%time
from folktexts.acs.acs_dataset import ACSDataset
acs_dataset = ACSDataset.make_from_task(
    task=task,
    subsampling=0.1,           # NOTE: optionally, use subsampling for faster but noisier results
    cache_dir=DATA_DIR,)

Loading ACS data...
CPU times: user 47.3 s, sys: 19.4 s, total: 1min 6s
Wall time: 1min 8s


**Note:** This dataset will contain all samples in the ACS PUMS file, you can
now filter the `acs_dataset.data` if you want to use only a portion of the data 
(e.g, only people in California, or only people above 18 years old).

As an example, we'll filter the ACS data to contain only people above 18 years old.

In [9]:
print(f"Original number of samples: {len(acs_dataset.data)}")

acs_dataset.data = acs_dataset.data[acs_dataset.data["AGEP"] >= 18]
print(f"Parsed number of samples: {len(acs_dataset.data)}")

Original number of samples: 3236107
Parsed number of samples: 2580544


### Load LLM from huggingface or from local disk

In [10]:
MODEL_NAME_OR_PATH = "meta-llama/Meta-Llama-3-8B"

In [11]:
%%time
from folktexts.llm_utils import load_model_tokenizer
model, tokenizer = load_model_tokenizer(MODEL_NAME_OR_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 4.21 s, sys: 7.41 s, total: 11.6 s
Wall time: 17.8 s


Create `LLMClassifier` object.

In [12]:
from folktexts.classifier import LLMClassifier

llm_clf = LLMClassifier(
    model=model,
    tokenizer=tokenizer,
    task=task,
    batch_size=20,
    context_size=600,
)

**Validation:** Check that model prompts with the custom task make sense, or if anything needs to be adjusted.

In [13]:
X_sample, _y_sample = acs_dataset.sample_n_train_examples(n=1)
print(llm_clf.encode_row(X_sample.iloc[0], question=llm_clf.task.question))

The following data corresponds to a survey respondent. The survey was conducted among US residents in 2018. Please answer the question based on the information provided. The data provided is enough to reach an approximate answer.

Information:
- The age is: 26 years old.
- The highest educational attainment is: 9th grade.
- The marital status is: Never married.
- The place of birth is: Georgia.
- The usual number of hours worked per week is: 45 hours.
- The sex is: Female.
- The race is: White.
- The resident state is: Georgia.
- The citizenship status is: Born in the United States.
- The disability status is: With a disability.
- The yearly income is: $20,000.

Question: Has this person ever served in the military?
A. Yes, this person is now on active duty or was in the past.
B. No, this person has never served in the military.
Answer:


### Run benchmark on the custom task

In [14]:
from folktexts.benchmark import Benchmark, BenchmarkConfig

bench = Benchmark(
    llm_clf=llm_clf,
    dataset=acs_dataset,
    config=BenchmarkConfig.default_config(),
)

In [15]:
acs_dataset.get_train()[1].value_counts()

MIL!=4
0    1864068
1     200367
Name: count, dtype: int64

In [16]:
%%time
bench.run(results_root_dir=".")

Computing risk estimates:   0%|          | 0/12903 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [ ]:
bench.plot_results();

ValueError: No results to plot. Run the benchmark first.